# Proyecto 1 - Visión por Computadora
## Ejercicio 2
## Integrantes:

- Javier Alvarado - 21188
- Mario Guerra - 21008
- Emilio Solano - 21212

In [1]:
import cv2
import numpy as np
import networkx as nx
import json
import os
from skimage.morphology import skeletonize
from scipy import ndimage

def cargar_imagen(ruta_imagen):
    """
    Carga una imagen groundtruth y la convierte a binaria.
    """
    img = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen: {ruta_imagen}")
    
    # Asegurarse de que la imagen sea binaria (0s y 1s)
    _, img_bin = cv2.threshold(img, 127, 1, cv2.THRESH_BINARY)
    return img_bin

def obtener_esqueleto(img_bin):
    """
    Obtiene el esqueleto de la imagen binaria usando skeletonize.
    """
    # Esqueletización con scikit-image
    esqueleto = skeletonize(img_bin)
    # Convertir a tipo uint8 (0 y 255)
    esqueleto = esqueleto.astype(np.uint8) * 255
    return esqueleto

def identificar_vertices_y_aristas(esqueleto):
    """
    Identifica vértices (puntos de bifurcación) y aristas (segmentos) en el esqueleto.
    """
    # Convertir a binario nuevamente (0 y 1)
    esqueleto_bin = esqueleto // 255
    
    # Crear un kernel 3x3 para contar vecinos
    kernel = np.ones((3, 3), dtype=np.uint8)
    kernel[1, 1] = 0  # Excluir el píxel central
    
    # Contar vecinos para cada píxel en el esqueleto
    vecinos = ndimage.convolve(esqueleto_bin, kernel, mode='constant', cval=0)
    
    # Localizar puntos de bifurcación (más de 2 vecinos) y puntos finales (1 vecino)
    puntos_bifurcacion = np.logical_and(esqueleto_bin == 1, vecinos > 2)
    puntos_finales = np.logical_and(esqueleto_bin == 1, vecinos == 1)
    
    # Obtener coordenadas de los puntos de bifurcación y puntos finales
    coords_bifurcacion = np.argwhere(puntos_bifurcacion)
    coords_finales = np.argwhere(puntos_finales)
    
    # Considerar tanto puntos de bifurcación como puntos finales como vértices
    vertices = np.vstack((coords_bifurcacion, coords_finales))
    
    # Crear grafo
    G = nx.Graph()
    
    # Agregar nodos al grafo (índice: coordenadas)
    for i, (y, x) in enumerate(vertices):
        G.add_node(i, pos=(int(y), int(x)))
    
    # Trazar aristas entre vértices siguiendo el esqueleto
    for i, (y1, x1) in enumerate(vertices):
        for j, (y2, x2) in enumerate(vertices):
            if i != j:
                # Trazar una línea entre dos vértices y verificar si sigue el esqueleto
                es_arista, puntos_arista = verificar_arista(esqueleto_bin, (y1, x1), (y2, x2))
                if es_arista:
                    G.add_edge(i, j, points=puntos_arista)
    
    return G

def verificar_arista(esqueleto, punto1, punto2):
    """
    Verifica si hay una arista válida entre dos puntos siguiendo el esqueleto.
    Utiliza una versión simplificada de trazado de línea para verificación.
    """
    y1, x1 = punto1
    y2, x2 = punto2
    
    # Distancia Manhattan entre los puntos
    distancia = abs(y2 - y1) + abs(x2 - x1)
    
    # Si los puntos están demasiado lejos, no hay arista directa
    if distancia > 20:  # Umbral ajustable
        return False, []
    
    # Trazar línea de Bresenham entre los puntos
    puntos_linea = trazar_linea(y1, x1, y2, x2)
    
    # Verificar que la mayoría de los puntos de la línea estén en el esqueleto
    puntos_en_esqueleto = [p for p in puntos_linea if 0 <= p[0] < esqueleto.shape[0] and 
                          0 <= p[1] < esqueleto.shape[1] and esqueleto[p[0], p[1]] == 1]
    
    # Si al menos el 90% de los puntos están en el esqueleto, es una arista válida
    if len(puntos_en_esqueleto) > 0.9 * len(puntos_linea):
        return True, puntos_linea
    
    return False, []

def trazar_linea(y1, x1, y2, x2):
    """
    Implementación simple del algoritmo de Bresenham para trazar una línea entre dos puntos.
    Retorna las coordenadas de los puntos que forman la línea.
    """
    puntos = []
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    sx = 1 if x1 < x2 else -1
    sy = 1 if y1 < y2 else -1
    err = dx - dy
    
    x, y = x1, y1
    
    while True:
        puntos.append((y, x))
        if x == x2 and y == y2:
            break
        e2 = 2 * err
        if e2 > -dy:
            err -= dy
            x += sx
        if e2 < dx:
            err += dx
            y += sy
    
    return puntos

def guardar_grafo_como_json(G, ruta_salida):
    """
    Guarda el grafo como un archivo JSON con la estructura requerida.
    """
    # Crear diccionario con la estructura del grafo
    grafo_dict = {
        "nodos": [],
        "aristas": []
    }
    
    # Agregar nodos al diccionario
    for nodo in G.nodes():
        pos = G.nodes[nodo]['pos']
        grafo_dict["nodos"].append({
            "id": int(nodo),
            "coordenadas": {"fila": int(pos[0]), "columna": int(pos[1])}
        })
    
    # Agregar aristas al diccionario
    for u, v in G.edges():
        grafo_dict["aristas"].append({
            "origen": int(u),
            "destino": int(v)
        })
    
    # Guardar en archivo JSON
    with open(ruta_salida, 'w') as f:
        json.dump(grafo_dict, f, indent=4)
    
    print(f"Grafo guardado en {ruta_salida}")

def visualizar_grafo(imagen_original, G, ruta_salida=None):
    """
    Visualiza el grafo sobre la imagen original y guarda el resultado.
    """
    # Crear una imagen a color para visualización
    img_color = cv2.cvtColor(imagen_original * 255, cv2.COLOR_GRAY2BGR)
    
    # Dibujar aristas
    for u, v in G.edges():
        y1, x1 = G.nodes[u]['pos']
        y2, x2 = G.nodes[v]['pos']
        cv2.line(img_color, (x1, y1), (x2, y2), (0, 0, 255), 1)
    
    # Dibujar vértices
    for node in G.nodes():
        y, x = G.nodes[node]['pos']
        cv2.circle(img_color, (x, y), 3, (0, 255, 0), -1)
    
    # Guardar la imagen si se proporciona una ruta
    if ruta_salida:
        cv2.imwrite(ruta_salida, img_color)
        print(f"Visualización guardada en {ruta_salida}")
    
    return img_color

def procesar_imagen(ruta_imagen, ruta_salida_json, ruta_salida_vis=None):
    """
    Función principal que procesa una imagen y genera el grafo.
    """
    # Cargar imagen
    img_bin = cargar_imagen(ruta_imagen)
    
    # Obtener esqueleto
    esqueleto = obtener_esqueleto(img_bin)
    
    # Identificar vértices y aristas
    G = identificar_vertices_y_aristas(esqueleto)
    
    # Guardar grafo como JSON
    guardar_grafo_como_json(G, ruta_salida_json)
    
    # Visualizar y guardar resultado si se proporciona una ruta
    if ruta_salida_vis:
        visualizar_grafo(img_bin, G, ruta_salida_vis)
    
    return G

def procesar_todas_imagenes(directorio_entrada, directorio_salida):
    """
    Procesa todas las imágenes groundtruth en el directorio especificado.
    """
    # Crear directorio de salida si no existe
    if not os.path.exists(directorio_salida):
        os.makedirs(directorio_salida)
    
    # Procesar cada imagen groundtruth
    for i in range(1, 21):
        nombre_archivo = f"{i}_gt.pgm"
        ruta_imagen = os.path.join(directorio_entrada, nombre_archivo)
        
        # Verificar si el archivo existe
        if os.path.exists(ruta_imagen):
            print(f"Procesando {nombre_archivo}...")
            
            # Rutas de salida
            ruta_salida_json = os.path.join(directorio_salida, f"{i}_grafo.json")
            ruta_salida_vis = os.path.join(directorio_salida, f"{i}_visualizacion.png")
            
            # Procesar imagen
            procesar_imagen(ruta_imagen, ruta_salida_json, ruta_salida_vis)
        else:
            print(f"No se encontró el archivo {ruta_imagen}")

# Ejemplo de uso
if __name__ == "__main__":
    directorio_entrada = "./data/database/"
    directorio_salida = "./resultados/"
    
    # Procesar todas las imágenes
    procesar_todas_imagenes(directorio_entrada, directorio_salida)
    
    # Alternativamente, procesar una sola imagen
    # procesar_imagen("./data/database/1_gt.pgm", "./resultados/1_grafo.json", "./resultados/1_visualizacion.png")

Procesando 1_gt.pgm...
Grafo guardado en ./resultados/1_grafo.json
Visualización guardada en ./resultados/1_visualizacion.png
Procesando 2_gt.pgm...
Grafo guardado en ./resultados/2_grafo.json
Visualización guardada en ./resultados/2_visualizacion.png
Procesando 3_gt.pgm...
Grafo guardado en ./resultados/3_grafo.json
Visualización guardada en ./resultados/3_visualizacion.png
Procesando 4_gt.pgm...
Grafo guardado en ./resultados/4_grafo.json
Visualización guardada en ./resultados/4_visualizacion.png
Procesando 5_gt.pgm...
Grafo guardado en ./resultados/5_grafo.json
Visualización guardada en ./resultados/5_visualizacion.png
Procesando 6_gt.pgm...
Grafo guardado en ./resultados/6_grafo.json
Visualización guardada en ./resultados/6_visualizacion.png
Procesando 7_gt.pgm...
Grafo guardado en ./resultados/7_grafo.json
Visualización guardada en ./resultados/7_visualizacion.png
Procesando 8_gt.pgm...
Grafo guardado en ./resultados/8_grafo.json
Visualización guardada en ./resultados/8_visualizac